In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

In [ ]:
!nvidia-smi
!pip install -U git+https://github.com/qubvel/efficientnet

In [ ]:
%tensorflow_version 2.x

from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.utils import to_categorical, plot_model
import tensorflow.keras.callbacks as cb
import tensorflow.keras.layers as L
import tensorflow.keras as K
import tensorflow as tf

import efficientnet.tfkeras as efn
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm
from glob import glob
import os, zipfile, cv2

# 載入資料

In [ ]:
IMAGE_SIZE = 224
BATCH_SIZE = 16
NUM_EPOCHS = 10
STEPS = 128
NUM_CLASSES = 3
DATASET_PATH = '/content/datasets/'

In [ ]:
if not os.path.exists('datasets'):
    f = zipfile.ZipFile("/content/drive/My Drive/Colab Notebooks/datasets-300.zip")
    f.extractall("./")

# CNN、FC 模型建立

In [ ]:
def convert_to_numpy(x_data, rescale = 1.):
    class_map = {'A': 0, 'B': 1, 'C': 2}
    train_datas = np.zeros((len(x_data), IMAGE_SIZE, IMAGE_SIZE, 3), dtype=np.float32)
    train_label = np.zeros(len(x_data), dtype=np.float32)

    for i, path in enumerate(x_data):
        img = load_img(path, target_size=(IMAGE_SIZE, IMAGE_SIZE))
        img = img_to_array(img)
        train_datas[i] = img * rescale
        
        c = path.split('/')[-2]
        train_label[i] = class_map[c]
        
    return train_datas, train_label

x_train = glob('/content/datasets/train/*/*.jpg')
x_test = glob('/content/datasets/test/*/*.jpg')

np.random.shuffle(x_train)
np.random.shuffle(x_test)

x_train_np, y_train_np = convert_to_numpy(x_train)
x_test_np, y_test_np = convert_to_numpy(x_test, rescale= 1./255)

In [ ]:
trn_dagen = ImageDataGenerator(rescale = 1.0 / 255,
                               rotation_range=40,
                               width_shift_range=0.2,
                               height_shift_range=0.2,
                               shear_range=0.2,
                               zoom_range=0.2,
                               channel_shift_range=10,
                               horizontal_flip=True,
                               vertical_flip=True, 
                               brightness_range=[0.5, 1.5],
                               fill_mode='nearest')

trn_dagen.fit(x_train_np)

In [ ]:
net = efn.EfficientNetB7(input_shape= (x_train_np[0].shape),
                         weights= 'noisy-student',
                         include_top= False)

for layer in net.layers:
    layer.trainable = True

x = net.output
# BN-Relu
x = L.BatchNormalization()(x)
x = L.Activation('relu')(x)
x = L.GlobalAveragePooling2D()(x)
out = L.Dense(NUM_CLASSES, activation=tf.nn.softmax)(x)

model = Model(inputs=net.input, outputs=out)

258072576/258068648 [==============================] - 8s 0us/step


In [ ]:
model.compile(loss= K.losses.SparseCategoricalCrossentropy(), 
              optimizer= K.optimizers.Adam(), 
              metrics= [K.metrics.SparseCategoricalAccuracy()])

In [ ]:
LR_START = 0.0001
LR_MAX = 0.00005
LR_MIN = 0.0001
LR_RAMPUP_EPOCHS = 4
LR_SUSTAIN_EPOCHS = 6
LR_EXP_DECAY = .8

def lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
    return lr
    
lr_callback = cb.LearningRateScheduler(lrfn, verbose=True)

rl_callback = cb.ReduceLROnPlateau(monitor = "val_loss", 
                                   factor = 0.5, 
                                   patience = 10, 
                                   verbose = 0, 
                                   mode = "auto", 
                                   min_delta = 1e-07, 
                                   cooldown = 0, 
                                   min_lr = 1e-7)

es_callback = cb.EarlyStopping(patience=15, restore_best_weights=True)

# rng = [i for i in range(NUM_EPOCHS)]
# y = [lrfn(x) for x in rng]
# plt.plot(rng, y)
# print("Learning rate schedule: {:.3g} to {:.3g} to {:.3g}".format(y[0], max(y), y[-1]))

callbacks = [lr_callback, 
             rl_callback, 
             es_callback]

In [ ]:
history = model.fit(trn_dagen.flow(x_train_np, y_train_np, batch_size= BATCH_SIZE),
                    steps_per_epoch= len(x_train_np) // BATCH_SIZE,
                    epochs= NUM_EPOCHS,
                    verbose= 1,
                    validation_data=(x_test_np, y_test_np),
                    callbacks= callbacks)

# Plot accuracy

In [ ]:
plt.plot(history.history['sparse_categorical_accuracy'])
plt.plot(history.history['val_sparse_categorical_accuracy'])
plt.legend(['acc', 'val_acc'])
plt.title('acc')

# Plot loss 

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.title('loss')

# 評估模型

In [ ]:
from tensorflow.keras.models import load_model

e = model.evaluate(x_test_np, y_test_np)
e

In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix

pre = model.predict(x_test_np)
pre = pre.argmax(axis=-1)

df = pd.DataFrame(confusion_matrix(y_test_np, pre))
df